In [24]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [25]:
env = GridworldEnv()

In [49]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        V_next = np.copy(V)
        for s in range(env.nS):
            V_next[s] = 0
            for a, action_prob in enumerate(policy[s]):
                for prob, next_state, reward, done in env.P[s][a]:
                    V_next[s] += action_prob * prob * (reward + discount_factor * V[next_state])  
        
        print(f'Value functions: {V}')
        diff = np.abs(V - V_next)
        converged = diff < theta
        V = V_next
        if np.all(converged):
            break
    return np.array(V)

In [50]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

Value functions: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Value functions: [ 0. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  0.]
Value functions: [ 0.   -1.75 -2.   -2.   -1.75 -2.   -2.   -2.   -2.   -2.   -2.   -1.75
 -2.   -2.   -1.75  0.  ]
Value functions: [ 0.     -2.4375 -2.9375 -3.     -2.4375 -2.875  -3.     -2.9375 -2.9375
 -3.     -2.875  -2.4375 -3.     -2.9375 -2.4375  0.    ]
Value functions: [ 0.      -3.0625  -3.84375 -3.96875 -3.0625  -3.71875 -3.90625 -3.84375
 -3.84375 -3.90625 -3.71875 -3.0625  -3.96875 -3.84375 -3.0625   0.     ]
Value functions: [ 0.        -3.65625   -4.6953125 -4.90625   -3.65625   -4.484375
 -4.78125   -4.6953125 -4.6953125 -4.78125   -4.484375  -3.65625
 -4.90625   -4.6953125 -3.65625    0.       ]
Value functions: [ 0.         -4.20898438 -5.50976562 -5.80078125 -4.20898438 -5.21875
 -5.58984375 -5.50976562 -5.50976562 -5.58984375 -5.21875    -4.20898438
 -5.80078125 -5.50976562 -4.20898438  0.        ]
Value functions: [

In [51]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)